This is a black jack game developed as a homework assigment in my Udemy Python course. It can have more players, and players moving in and out from the table. The only moves are hit or stand, other moves can be developed later.

In [45]:
# Initiating modules and Cards suits, rankes and values:

import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

#Defining classes used in the gameplay
class Card:
    '''
    The Card class initiates the cards, every card has a suit, a rank and a value.
    You can print out a card, showint its suit and rank.
    '''
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return f"{self.rank} of {self.suit}"


class Deck:
    '''
    The Deck class initiates a new Deck, which you can shuffle and draw a card from. 
    You can print out the deck, showing the cards in it in order.
    You can evaluate, how many cards left in the deck with len()
    '''
    def __init__(self):
        self.all_cards=[]
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Card(suit,rank))        
    
    #method for shuffling the deck
    def shuffle(self):
        random.shuffle(self.all_cards)

    #method for dealing the next card from the deck
    def deal_one(self):
        return self.all_cards.pop()

    #method for printing the deck    
    def __str__(self):
        self.cardlist = ''
        num = 0
        for c in self.all_cards:
            num +=1
            self.cardlist += f"  {num}. card: {c.rank} of {c.suit}\n"
        return f'Cards in the Deck:\n{self.cardlist}'
   
    #method for counting the cards in the deck  
    def __len__(self):
        return len(self.all_cards)

class Player:
    '''
    The Player class initiates a PLAYER, whith a name and with zero cards, 
    and statistics about their previous games (0 games inistially) and their balance (chips they start with) and gain (also 0).
    The eval_hand method computes the total value of cards in the hand of the player, the number of aces, 
    the status of the player and the best value using the aces as 11 or 1 accordingly.
    
    Printing the playes shows what cards he/she holds and gives back the total value of their cards.
    '''
    
    def __init__(self, name, chips):
        self.name = name
        self.all_cards = []
        self.statistics = {"wins": 0, "losses": 0, "ties": 0, "balance": chips, "gain": 0}

    #method for evaluating the cards of the player    
    def eval_hand(self):
        
        self.value_all = 0 #the total value of cards, aces are counted as 11
        self.num_aces=0 #number of aces
        self.status='bust' #status, can be in_play (with a value below 17), stop_zone (between 17 and 20),
                            # blackjack (21 from two cards), 21 (21 with more than two cards ) and 
                            # bust (value exceeds 21).
                            # Aces are counted as 1 or 11 to reach blackjack, or 21 as the next best option, 
                            # or stop_zone as the next best option, or at least play_zone 
                        
        self.value_hand=0 #The value hand with the ideal ace-values (bust is evaluated as 0 and blackjack as 22)
        
        for c in self.all_cards:
            self.value_all += c.value
            
            if c.rank == 'Ace':
                self.num_aces += 1
        
        for i in range(self.num_aces,-1,-1): #we go from counting all aces as 1 to counting all aces as 11
 
            if self.value_all - i*10 == 21:  #if we achieve blackjack (two cards, one is ace) or 21, we stop
                if len(self.all_cards) == 2:
                    self.status = 'blackjack'
                    self.value_hand=22
                    break
                else:
                    self.status = '21'
                    self.value_hand=21
                    break
                
            elif 17 <= self.value_all - i*10 < 21: #if we reach stop_zone with a combination of ace values, we stop, 
                                                    # no 21 can be reached any more
                self.status = 'stop_zone'
                self.value_hand=self.value_all - i*10
                break
                
            elif self.value_all - i*10 < 17: #if we can achieve a value less than 17, we choose that as hand value 
                                             # unless the hand value is already bigger than that
                self.status = 'in_play'
                if self.value_hand < self.value_all - i*10:
                    self.value_hand = self.value_all - i*10
                
        #if we cannot assign any of the statuses above, the hand status stays bust and the hand value remains 0
                    
        return self.value_all, self.num_aces, self.status, self.value_hand
 
    #method for printing the cards of the player    
    def __str__(self):
        
        self.card_list=''
        self.value_hand_variants=str(self.value_all)
        
        for c in self.all_cards:
            self.card_list += f"   {c.rank} of {c.suit}\n"
            
        if self.num_aces > 0:
            for i in range(self.num_aces):
                self.value_hand_variants += f' or {self.value_all - (i+1)*10}'
            
        return f'\n{self.name} has {len(self.all_cards)} cards:\n{self.card_list}\nThe value of card(s) is {self.value_hand_variants}\n' 

    #method for adding the prize to the balance and gain of the player. Prize for Blackjack is 1.5 times the bet
    def win_bet(self, bet):
        if self.status == 'blackjack':
            prize = bet + bet//2
        else:
            prize = bet
            
        self.statistics['balance'] += prize
        self.statistics['gain'] += prize

    #method for substracting the bet from the balance and gain in case of a lost game
    def lose_bet(self, bet):
        self.statistics['balance'] -= bet
        self.statistics['gain'] -= bet
        

In [46]:
#defining funcions

def ask_for_new_players(num_of_current_players):

    '''
    Asking how many new players do we have. 
    The total amount of players should be between 0 and 5, the function checks it and asks again if it is not the case
    '''
    total = 'WRONG'
    while total.isdigit() == False:

        total=input("Please let us know how many players are going to play (0 to 5): ")
        
        if total.isdigit() == False:
            print("Please provide a positive whole number!")
        else:
            if int(total) < 0 or int(total) + num_of_current_players > 5:
                print("Please provide a whole number between 0 and 5!")
                total="WRONG"
            
    return int(total)


def ask_for_chips():

    '''
    Asking the player how many chips he/she has. 
    Should provide a positive whole number, the funcion checks it and asks again if it is not the case
    '''
    total = 'WRONG'
    while total.isdigit()==False:

        total=input("Please let us know how many chips do you have to play with: ")
        
        if total.isdigit()==False:
            print("Please provide a positive whole number!")
        else:
            if int(total)<=0:
                print("Please provide a positive whole number!")
                total="WRONG"
            
    return int(total)

def ask_for_bet(player):

    '''
    Asking the player how many chips he/she wants to risk in the next round. 
    Should provide a positive whole number, the funcion checks it and asks again if it is not the case
    Also, should provide a bet less then the amount of his/her total chips
    '''
    bet = 'WRONG'
    
    while bet.isdigit()==False:
        bet=input(f"{player.name}, what is your bet for the next game? You have {player.statistics['balance']} chips in total. ")
        
        if bet.isdigit()==False:
            print("Please provide a positive whole number!")
        else:
            if int(bet)<=0:
                print("Please provide a positive whole number!")
                bet="WRONG"
            elif int(bet) > player.statistics['balance']:
                print(f"{player.name}, you have less chips than that ({player.statistics['balance']} in total)")
                bet="WRONG"
                
    return int(bet)

def player_action(player):
    '''
    Asking the player about his next move. 
    Should provide a valid answer, the funcion checks it and asks again if it is not the case
    The player can move only until he/she reaches or exceeds 21.
    '''
    acceptable_moves=["hit", "stand"]
    move="wrong"
    
    while move.lower() not in acceptable_moves:
        move = input(f"{player.name}, what is your next move, hit or stand? Please choose (write hit or stand): " ).lower()
    
        if move not in acceptable_moves:
            print("Your choice is not valid, please provide a valid move!")
    
    return move
    
def winner(players, dealer, players_bet, final=1):
    '''
    Evaluates the game, decides who won, tied or lost. 
    There are two checks during the game: if the dealer has a blackjack, only players with also a blackjack can tie,
    in this case the first if branch runs.
    If it is not the case, at the first check the elif, and than the else branch runs.
    '''
    
    #evaluating the hands
    dealer.eval_hand()
    
    for i in range(len(players)): 
        players[i].eval_hand()
        
    if final==0 and dealer.status == 'blackjack': #if at the first check the dealer has a blackjack, the game is stopped
                                                    #and players are evaluated
        
        print(dealer)
        
        print("The dealer has a blackjack!")
        
        game_on = False
        
        for i in range(num_of_players): 
            
            if players[i].status=='blackjack':
                print(player)
                print(f"{players[i].name} has a black jack too, there is a tie")
                players[i].statistics['ties'] += 1
                
            else: 
                print(player)
                print(f"{players[i].name} lost the game")
                players[i].statistics['losses'] += 1
                players[i].lose_bet(players_bet[i])
                
    elif final==0: #else the game goes on with the players moves
        game_on=True
    
    else:  #at the final check every player is measured to the dealer
        game_on=False
        print(dealer)
        
        for i in range(num_of_players):
            
            if players[i].value_hand == 0:  #if the player is busted, they lost
                print(players[i])
                print(f"{players[i].name} busted and lost their bet")
                players[i].statistics['losses'] += 1
                players[i].lose_bet(players_bet[i])
            
            elif players[i].value_hand == 22: #if the player has a blackjack, they won (the dealer surely does not have one at this point)
                print(players[i])
                print(f"{players[i].name} won with a blackjack")
                players[i].statistics['wins'] += 1
                players[i].win_bet(players_bet[i])
                
            elif players[i].value_hand > dealer.value_hand: #otherwise, higher values win
                print(players[i])
                print(f"{players[i].name} won with a hand value of {players[i].value_hand}")
                players[i].statistics['wins'] += 1
                players[i].win_bet(players_bet[i])
                
            elif players[i].value_hand == dealer.value_hand: #ties get back the bet
                print(players[i])
                print(f"{players[i].name} has a tie with the dealer with a hand value of {players[i].value_hand}")
                players[i].statistics['ties'] += 1
            
            else: #lower values lose
                print(players[i])
                print(f"{players[i].name} lost the game with a hand value of {players[i].value_hand}")
                players[i].statistics['losses'] += 1
                players[i].lose_bet(players_bet[i])
        
    return game_on
    

def ask_for_playon(player):
    '''
    Asking the players whether they want to play another game. 
    Should provide a valid answer, the funcion checks it and asks again if it is not the case
    '''
    
    acceptable_answers=["y", "n"]
    answer="wrong"
    
    while answer.lower() not in acceptable_answers:
        answer = input(f"{player.name}, do you want to play another game (Y or N): " ).lower()
    
        if answer not in acceptable_answers:
            print("Your choice is not valid, please provide a valid answer!")
            
        if answer == 'n':
            print(f"{player.name}, thank you for the play!") 
            print(f"You have played {player.statistics['wins'] + player.statistics['ties']+player.statistics['losses']} games of which you won {player.statistics['wins']} and lost {player.statistics['losses']}.")
            print(f"Your final balance is {player.statistics['balance']} and your gain is {player.statistics['gain']}.")
    
    return answer == 'y'

In [ ]:
#initiating the game
play_on = True


#Informing the players about the game
print("Wellcome to our Black Jack game!")
print("You will play against or computer dealer as in a Casino")


#initiating players and their chips
players = []
num_of_players = ask_for_new_players(0)

if num_of_players == 0:
    play_on = False
    
for i in range(num_of_players):
    name = input(f"Player {i+1}., please let us know your name: ")
    chips_total = ask_for_chips()
    players.append(Player(name, chips_total))

dealer = Player("Dealer", 1)

#as long, as we have players, we play
while play_on:
    #initiating the Deck
    play_deck = Deck()
    play_deck.shuffle()
    
    #placing the bet and starting the game
    players_bet=[]
    for i in range(num_of_players):
        players_bet.append(ask_for_bet(players[i]))
        
    dealer_bet = 0
    
    game_on = True

    #empty the hands of players and deal the first two cards
    dealer.all_cards = []
    for i in range(num_of_players):
        players[i].all_cards = []
        
    for i in range(2):
        for i in range(num_of_players):
            players[i].all_cards.append(play_deck.deal_one())
            
        dealer.all_cards.append(play_deck.deal_one())
        
    #Show players' cards and one of the dealer cards to the players
    for i in range(num_of_players):
        players[i].eval_hand()
        print(players[i])

    dealer.eval_hand()
    print(f"\nThe first card of the dealer is {dealer.all_cards[0]}")
    
    #Check if the game is over already (if the dealer has a blackjack) and evvaluate the results is it is the case
    game_on = winner(players, dealer, players_bet, 0)
    
    #If the game is not finished with the dealer having a blackjack, we can carry on with dealing cards for players
    if game_on:
        for player in players:
            #if the player is in a decisional position (not reached 21 or busted, we can ask their next step)
            if player.status in ['in_play','stop_zone']:
                move='hit'
            else:
                move='stand'
                
            while move=='hit' and player.status in ['in_play','stop_zone']:
                move=player_action(player)
                
                if move == 'hit':
                    player.all_cards.append(play_deck.deal_one())
                    player.eval_hand()
                    print(player)
    
    
        #dealing for the dealer:
        while dealer.status == 'in_play':
            dealer.all_cards.append(play_deck.deal_one())
            dealer.eval_hand()
            
        #evaluate the final hands 
        game_on = winner(players, dealer, players_bet, 1)
        
    #asking players who wants to play a new game and filter the players list accordingly
    players = list(filter(lambda x: (ask_for_playon(x)), players))
    
    #adding new players to the table if there are new candidates. 
    num_of_new_players = ask_for_new_players(len(players))
    num_of_players = num_of_new_players + len(players)
    
    #If everyone opted out and no new players either, then the play is finished
    if len(players) + num_of_new_players==0:
        play_on=False
    
    #initiating new players and their chips
    for i in range(num_of_new_players):
        name = input("Please let us know your name: ")
        chips_total = ask_for_chips()
        players.append(Player(name, chips_total))
    
    
